<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_6_3__Swarm_raportowanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-wz502diw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-wz502diw
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.7 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=b287422edf09a

Ten kod instaluje pakiet "swarm" bezpośrednio z repozytorium GitHub należącego do OpenAI.

Polecenie !pip install służy do instalacji pakietów Pythona. Przedrostek ! oznacza, że komenda jest wykonywana w środowisku powłoki systemowej, a nie w samym Pythonie. Jest to częsta praktyka w notebookach Jupyter.

git+ na początku adresu URL wskazuje, że instalowany pakiet pochodzi z repozytorium Git. Dzięki temu pip wie, że ma pobrać kod źródłowy bezpośrednio z GitHuba, a nie z Python Package Index (PyPI).

https://github.com/openai/swarm.git to adres repozytorium zawierającego kod źródłowy pakietu. Po zainstalowaniu będzie można używać funkcjonalności pakietu swarm w kodzie Python poprzez instrukcję import.

In [ ]:
from swarm import Agent
from swarm.repl import run_demo_loop
import os
from google.colab import userdata


Ten kod importuje niezbędne komponenty do pracy z biblioteką swarm oraz zawiera dodatkowe importy:

`from swarm import Agent` - importuje klasę Agent z pakietu swarm, która służy do tworzenia i zarządzania agentami w systemie.

`from swarm.repl import run_demo_loop` - importuje funkcję run_demo_loop z modułu repl w pakiecie swarm. REPL to skrót od Read-Eval-Print Loop, czyli pętla odczyt-wykonanie-wyświetlenie. Ta funkcja prawdopodobnie obsługuje interaktywną demonstrację działania agentów.

`import os` - importuje moduł os, który zapewnia funkcje do interakcji z systemem operacyjnym, np. operacje na plikach i katalogach.

`from google.colab import userdata` - importuje moduł userdata z google.colab, który umożliwia dostęp do danych użytkownika w środowisku Google Colab.

In [ ]:

os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')


# Functions

In [ ]:
def complete_blog_post(title, content):
    # Create a valid filename from the title
    filename = title.lower().replace(" ", "-") + ".md"

    with open(filename, "w", encoding="utf-8") as file:
        file.write(content)

    print(f"Blog post '{title}' has been written to {filename}")
    return "Task completed"

In [ ]:
def admin_instructions(context_variables):
    topic = context_variables.get("topic", "No topic provided")
    return f"""You are the Admin Agent overseeing the blog post project on the topic: '{topic}'.
Your responsibilities include initiating the project, providing guidance, and reviewing the final content.
Once you've set the topic, call the function to transfer to the planner agent."""


def planner_instructions(context_variables):
    topic = context_variables.get("topic", "No topic provided")
    return f"""You are the Planner Agent. Based on the following topic: '{topic}'
Organize the content into topics and sections with clear headings that will each be individually researched as points in the greater blog post.
Once the outline is ready, call the researcher agent. """


def researcher_instructions(context_variables):
    return """You are the Researcher Agent. your task is to provide dense context and information on the topics outlined by the previous planner agent.
This research will serve as the information that will be formatted into a body of a blog post. Provide comprehensive research like notes for each of the sections outlined by the planner agent.
Once your research is complete, transfer to the writer agent"""


def writer_instructions(context_variables):
    return """You are the Writer Agent. using the prior information write a clear blog post following the outline from the planner agent.
    Summarise and include as much information relevant from the research into the blog post.
    The blog post should be quite large as the context the context provided should be quite dense.
Write clear, engaging content for each section.
Once the draft is complete, call the function to transfer to the Editor Agent."""


def editor_instructions(context_variables):
    return """You are the Editor Agent. Review and edit th prior blog post completed by the writer agent.
Make necessary corrections and improvements.
Once editing is complete, call the function to complete the blog post"""


In [ ]:
def transfer_to_researcher():
    return researcher_agent


def transfer_to_planner():
    return planner_agent


def transfer_to_writer():
    return writer_agent


def transfer_to_editor():
    return editor_agent


def transfer_to_admin():
    return admin_agent


def complete_blog():
    return "Task completed"


In [ ]:

def biegnij_forrest():
    run_demo_loop(admin_agent, debug=True)

# Wyroiło się

In [ ]:

admin_agent = Agent(
    name="Admin Agent",
    instructions=admin_instructions,
    functions=[transfer_to_planner],
)

planner_agent = Agent(
    name="Planner Agent",
    instructions=planner_instructions,
    functions=[transfer_to_researcher],
)

researcher_agent = Agent(
    name="Researcher Agent",
    instructions=researcher_instructions,
    functions=[transfer_to_writer],
)

writer_agent = Agent(
    name="Writer Agent",
    instructions=writer_instructions,
    functions=[transfer_to_editor],
)

editor_agent = Agent(
    name="Editor Agent",
    instructions=editor_instructions,
    functions=[complete_blog_post],
)

In [ ]:
biegnij_forrest()

Starting Swarm CLI 🐝
User: Impact of LLM on the legal profession
[2024-12-03 20:14:03] Getting chat completion for...: [{'role': 'system', 'content': "You are the Admin Agent overseeing the blog post project on the topic: 'No topic provided'.\nYour responsibilities include initiating the project, providing guidance, and reviewing the final content.\nOnce you've set the topic, call the function to transfer to the planner agent."}, {'role': 'user', 'content': 'Impact of LLM on the legal profession'}]
[2024-12-03 20:14:04] Received completion: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OO2LMVn3SilrBMf317skxn0W', function=Function(arguments='{}', name='transfer_to_planner'), type='function')])
[2024-12-03 20:14:04] Processing tool call: transfer_to_planner with arguments {}
[2024-12-03 20:14:04] Getting chat completion for...: [{'role': 'system', 'content': "You are the Planner Agent

EOFError: 

In [ ]:
!ls

effective-strategies-for-generative-ai-in-enterprise.md  sample_data
impact-of-llm-on-the-legal-profession.md
